In [1]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

  Using cached langgraph-0.2.67-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_anthropic-0.3.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached langgraph_checkpoint-2.0.10-py3-none-any.whl.metadata (4.6 kB)
  Using cached langgraph_sdk-0.1.51-py3-none-any.whl.metadata (1.8 kB)
  Using cached anthropic-0.45.0-py3-none-any.whl.metadata (23 kB)
  Using cached aiosqlite-0.20.0-py3-none-any.whl.metadata (4.3 kB)
Using cached langgraph-0.2.67-py3-none-any.whl (146 kB)
Using cached langchain_anthropic-0.3.4-py3-none-any.whl (22 kB)
Using cached aiosqlite-0.20.0-py3-none-any.whl (15 kB)
Using cached anthropic-0.45.0-py3-none-any.whl (222 kB)
Using cached langgraph_checkpoint-2.0.10-py3-none-any.whl (37 kB)
Using cached langgraph_sdk-0.1.51-py3-none-any.whl (44 kB)
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.3
    Uninstalling msgpack-1.0.3:
      Successfully uninstalled msgpack-1.0.3
Note: you may need to restart the kernel to use updated packages

In [3]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
#search_results = search.invoke("what is the weather in SF")
search_results = search.invoke("What is a mortgage?")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.consumerfinance.gov/ask-cfpb/what-is-a-mortgage-en-99/', 'content': 'A mortgage is a loan that uses your home as collateral. Learn how to compare different mortgages, afford your monthly payment, and avoid risky features.'}, {'url': 'https://www.nerdwallet.com/article/mortgages/what-is-a-mortgage', 'content': "A mortgage is a loan used to buy a home. You repay the loan, with interest, over a set number of years. The property serves as collateral, meaning if you don't pay, the lender can take the home."}]


In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [8]:
model_with_tools = model.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_8pHY2PPN21dOYeJ6NyUALeCk', 'type': 'tool_call'}]


In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [13]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='299eb495-e2a6-4e7b-976d-71ecee490591'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f3d91be6-059d-4ada-aa35-f6c86fdb4bc7-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [15]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='ca07eefd-b59a-494c-a609-750c50ce4b54'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bbwD8CznZ35mImPkeumCUvlL', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 88, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c4b79b3d-a4dd-427c-8d97-878c967ed001-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_bbwD8CznZ35mImPkeumCUvlL

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x0U5JpGxLQmLBJ32CrseFvH1', 'function': {'arguments': '{\n"query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ccdbaa7b-331b-4878-a07f-5028e695a4e4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_x0U5JpGxLQmLBJ32CrseFvH1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111, 'input_token_

In [17]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [18]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----") 

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6fcc357a-10c6-4d5d-940f-3b2d5cb92afd-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. How can I assist you further?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 108, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-33348108-ffca-4ed7-a018-aca6dcd81c9f-0', usage_metadata={'input_tokens': 108, 'output_tokens': 14, 'total_tokens': 122, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the 80/20 rule")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="The 80/20 rule, also known as the Pareto Principle, is a concept in economics that suggests 80% of outcomes (outputs) come from 20% of causes (inputs). Originally, it was observed by Vilfredo Pareto, an Italian economist, who noted in 1906 that 80% of the land in Italy was owned by 20% of the population.\n\nIt's important to note that the percentages aren't always exact, it might be 70/30 or 90/10 in some cases, but the principle holds that the majority of results come from a minority of causes. \n\nThis principle is applied across various fields like business management, where it's often observed that 80% of a company's profits come from 20% of its customers, or 80% of complaints come from 20% of customers. Additionally, it can be applied to personal productivity, where it's often found that 20% of the tasks you do result in 80% of your productivity. \n\nRemember, it's a principle or a rule-of-thumb rather than an exact mathematical law.", ad

In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What did I just ask you")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You just asked me about the 80/20 rule, also known as the Pareto Principle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 377, 'total_tokens': 399, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ed8afa2-501f-4ee4-b9b4-5374107165d7-0', usage_metadata={'input_tokens': 377, 'output_tokens': 22, 'total_tokens': 399, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
